In [157]:
# -*- coding: utf-8 -*-
#Tanasorn Chindasook 
#Web crawler + Google Sheets updater for apartment hunting 

from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
from pprint import pprint

#GoogleSheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [158]:
# URL or List of URLS
URLS = ['https://www.immobilienscout24.de/expose/119164572?referrer=RESULT_LIST_MAP&navigationServiceUrl=%2FSuche%2Fcontroller%2FexposeNavigation%2Fnavigate.go%3FsearchUrl%3D%2FSuche%2Fradius%2Fwohnung-mieten%3Fcenterofsearchaddress%253DBerlin%3B%3B%3B1276003001046%3B%3BMitte%252520%28Mitte%29%2526price%253D1000.0-2000.0%2526livingspace%253D50.0-%2526geocoordinates%253D52.5234%3B13.41139%3B4.0%2526pagenumber%253D2%2526enteredFrom%253Dresult_list%2526enterFromNavigationBar%253Dtrue%2526navigation%253Dzurueck%2526viewMode%253DMAP%26exposeId%3D119164572&navigationHasPrev=true&navigationHasNext=true&navigationBarType=RESULT_LIST&searchId=db713351-2bc4-3860-aebb-18699e5ec4ba&resultListViewMode=MAP&boundingBox=52.519471%2C13.378345%2C52.53853%2C13.42641&searchType=radius#/',
       'https://www.immobilienscout24.de/expose/117627923?referrer=RESULT_LIST_MAP&navigationServiceUrl=%2FSuche%2Fcontroller%2FexposeNavigation%2Fnavigate.go%3FsearchUrl%3D%2FSuche%2Fradius%2Fwohnung-mieten%3Fcenterofsearchaddress%253DBerlin%3B%3B%3B1276003001046%3B%3BMitte%252520%28Mitte%29%2526price%253D1000.0-2000.0%2526livingspace%253D50.0-%2526geocoordinates%253D52.5234%3B13.41139%3B4.0%2526enteredFrom%253Dresult_list%2526viewMode%253DMAP%26exposeId%3D117627923&navigationHasPrev=true&navigationHasNext=true&navigationBarType=RESULT_LIST&searchId=5f5f6faf-4db1-3096-9f15-6a4473e94ef0&resultListViewMode=MAP&boundingBox=52.522292%2C13.378345%2C52.535711%2C13.42641&searchType=radius#/',
       'https://www.immobilienscout24.de/expose/119447684?referrer=RESULT_LIST_MAP&navigationServiceUrl=%2FSuche%2Fcontroller%2FexposeNavigation%2Fnavigate.go%3FsearchUrl%3D%2FSuche%2Fradius%2Fwohnung-mieten%3Fcenterofsearchaddress%253DBerlin%3B%3B%3B1276003001046%3B%3BMitte%252520%28Mitte%29%2526price%253D1000.0-2000.0%2526livingspace%253D50.0-%2526geocoordinates%253D52.5234%3B13.41139%3B4.0%2526pagenumber%253D2%2526enteredFrom%253Dresult_list%2526enterFromNavigationBar%253Dtrue%2526navigation%253Dzurueck%2526viewMode%253DMAP%26exposeId%3D119447684&navigationHasPrev=true&navigationHasNext=true&navigationBarType=RESULT_LIST&searchId=db713351-2bc4-3860-aebb-18699e5ec4ba&resultListViewMode=MAP&boundingBox=52.51962%2C13.368761%2C52.538679%2C13.416826&searchType=radius#/']

In [159]:
# GOOGLE SHEET KEY
GSKEY = '1ffjHfhzP9IHINpNJcKBOsDLRA2lSWA5VrcreDNe-o30'

In [160]:
# Connect to Google Sheets 

scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json',scope)
client = gspread.authorize(creds)

sheet = client.open_by_key(GSKEY).sheet1

In [163]:
# BeautifulSoup HTML retrieval
for URL in URLS:
    r = requests.get(URL, verify=False)
    page_html = r.text
    page_soup = soup(page_html,'html.parser')

    # Get variables from webpage to insert
    flat_name = page_soup.find('h1').text

    address_1 = page_soup.find('span',{'class':'block font-nowrap print-hide'}).text
    address_2 = page_soup.find('span',{'class':'zip-region-and-country'}).text
    address = (address_1 + address_2).strip()

    rooms = int(page_soup.find('div',{'class':'is24qa-zi is24-value font-semibold'}).text.strip())

    sqm = float(page_soup.find('div',{'class':'is24qa-flaeche is24-value font-semibold'})\
    .text.strip()\
    .replace(',','.')\
    .split(' ')\
    [0])

    move_in_date = page_soup.find('dd',{'class':'is24qa-bezugsfrei-ab grid-item three-fifths'}).text.strip()
    
    try:
        floor = page_soup.find('dd',{'class':'is24qa-etage grid-item three-fifths'}).text.strip()
    except:
        floor = ''
    try:
        price_warm = float(page_soup.find('dd',{'class':'is24qa-gesamtmiete grid-item three-fifths font-bold'})\
        .text.strip()\
        .replace('.','')\
        .split(' ')\
        [0])
    except:
        price_warm = ''

    pps = float(price_warm)/float(sqm)
    
    # create row to update in gsheets
    insert_row = [flat_name, sqm, rooms, floor, price_warm, address, URL, '', '', '','','','','',pps, move_in_date]
    
    #insert row into gsheets
    sheet.insert_row(insert_row,16)

/Users/mimichindasook/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.immobilienscout24.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/mimichindasook/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.immobilienscout24.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/mimichindasook/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.immobilienscout24.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/